In [42]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import xmltodict, json
import matplotlib.pyplot as plt
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [43]:
def get_data(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'xml')
    data = xmltodict.parse(str(soup))
    return data

def get_obs_data(dataframe):
   x_dict = []
   y_dict = []
   for i in range(0,len(dataframe['Obs'])):
      x = dataframe['Obs'][i]['@TIME_PERIOD'] 
      y = dataframe['Obs'][i]['@OBS_VALUE']
      x = pd.to_datetime(x).strftime('%Y-%m') #convert x to datetime
      x_dict.append(x)
      y_dict.append(float(y)) #convert y to float
   return x_dict, y_dict

def make_gif(df):
    x, y = get_obs_data(df)
    filenames = []
    label = desc[desc['indicator'] == df['@INDICATOR']]['desc'].values[0]
    indicator = str(df_cpi['@INDICATOR'])
    for i in range(0, len(x)):
        fig, ax = plt.subplots(figsize=(10,5))
        plt.plot(x[:i], y[:i], label=label)
        plt.xlabel('Month')
        plt.xticks(rotation=90)
        plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
        #create file name and add to list
        filename = f'{i}.png'
        filenames.append(filename)
        #save frames
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
    # build gif
    gif_filename = f'{indicator}.gif'
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    duration = 0.15
    imageio.mimsave(gif_filename, images, duration=duration, loop=1)
    # Remove files
    for filename in set(filenames):
        os.remove(filename)

In [44]:
#database with url
database = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/thanhqtran/gso-macro-monitor/main/dsbb_database.csv')).reset_index(drop=True)
#convert data values to string 
database['url'].apply(lambda x: str(x))
database['var'].apply(lambda x: str(x))

#database description
desc = pd.DataFrame(pd.read_csv('https://raw.githubusercontent.com/thanhqtran/gso-macro-monitor/main/dsbb_indicator_desc.csv')).reset_index(drop=True)
desc['indicator'].apply(lambda x: str(x))
desc['desc'].apply(lambda x: str(x))
desc['domain'].apply(lambda x: str(x))
# do not display output 
pd.set_option('display.max_columns', None)

#directory 
directory = os.getcwd()
save_loc = f'{directory}/generated_gif/'

In [45]:
df_gdp = get_data(database['url'][1])
df_cpi = get_data(database['url'][2])
df_govbudget = get_data(database['url'][3])
df_gov_debt = get_data(database['url'][4])
df_interest = get_data(database['url'][5])
df_stock = get_data(database['url'][6])
df_bop = get_data(database['url'][7])
df_extdebt = get_data(database['url'][8])
df_exr = get_data(database['url'][9])
df_iip = get_data(database['url'][10])
df_lab = get_data(database['url'][11])
df_ppi = get_data(database['url'][12])
df_pop = get_data(database['url'][13])

In [35]:
df_cpi1 = df_cpi['message:StructureSpecificData']['message:DataSet']['Series']
x = get_obs_data(df_cpi1[0])[0]

# CPI components
filenames = []
i_list = [1,5,6,7,8,9,11,12,13,15,16]
labels = []
ys = {}
for i in i_list:
    label_raw = desc[desc['indicator'] == df_cpi1[i]['@INDICATOR']]['desc'].values[0]
    label = label_raw[0:20]
    labels.append(label)
    ys[label] = get_obs_data(df_cpi1[i])[1]
for time in range(0, len(date)):
    fig, ax = plt.subplots(figsize=(10,5))
    plt.plot(x[:time], ys[labels[0]][:time], label=labels[0])
    plt.plot(x[:time], ys[labels[1]][:time], label=labels[1])
    plt.plot(x[:time], ys[labels[2]][:time], label=labels[2])
    plt.plot(x[:time], ys[labels[3]][:time], label=labels[3])
    plt.plot(x[:time], ys[labels[4]][:time], label=labels[4])
    plt.plot(x[:time], ys[labels[5]][:time], label=labels[5])
    plt.plot(x[:time], ys[labels[6]][:time], label=labels[6])
    plt.plot(x[:time], ys[labels[7]][:time], label=labels[7])
    plt.plot(x[:time], ys[labels[8]][:time], label=labels[8])
    plt.plot(x[:time], ys[labels[9]][:time], label=labels[9])
    plt.plot(x[:time], ys[labels[10]][:time], label=labels[10])
    plt.xticks(rotation=45, size=10)
    plt.legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
    #plt.tight_layout()
    #create file name and add to list
    filename = f'{time}.png'
    filenames.append(filename)
    #save frames
    plt.title('Vietnam CPI components (index)')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
# build gif
gif_filename = f'{save_loc}cpi_component.gif'
images = []
duration = 0.15

with imageio.get_writer(gif_filename, mode='I', duration=duration, loop=1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)

# Main CPI index
cpi_agg = df_cpi1[0]
cpi_core = df_cpi1[32]
agg_x, agg_y = get_obs_data(cpi_agg)
core_x, core_y = get_obs_data(cpi_core)
label_agg = desc[desc['indicator'] == cpi_agg['@INDICATOR']]['desc'].values[0]
label_core = desc[desc['indicator'] == cpi_core['@INDICATOR']]['desc'].values[0]
# plot
filenames = []
for i in range(0, len(x)):
    fig, ax1 = plt.subplots(figsize=(10,5))
    ax1.bar(agg_x[:i], agg_y[:i], label=label_agg, alpha=0.3)
    ax1.set_ylabel(label_agg, color='b')
    ax1.tick_params(axis='x', labelrotation=90)
    ax2 = ax1.twinx()
    ax2.plot(core_x[:i], core_y[:i], label=label_core, color='r', marker='o')
    ax2.set_ylabel(label_core, color='r')
    fig.tight_layout()
    #create file name and add to list
    filename = f'{i}.png'
    filenames.append(filename)
    #save frames
    plt.title('Vietnam CPI Index and Core CPI change yoy')
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
# build gif
gif_filename = f'{save_loc}cpi.gif'
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
duration = 0.15
imageio.mimsave(gif_filename, images, duration=duration, loop=1)
# Remove files
for filename in set(filenames):
    os.remove(filename)